# Call Transcript Summarizer and Analyzer

## Problem Statement

* To Create a Python app/server which summarizes call transcript and analyzes sentiment
* Use Groq LLM model and API key
* Save the model's output in a .csv format  

## Transcripts

In [4]:
t1 = """Agent: Hello, thank you for calling. How can I help you today?
Customer: Hi, I tried booking a slot yesterday, but the payment kept failing.
Agent: I’m sorry to hear that. Did you receive any error message?
Customer: Yes, it said “transaction declined.” I was really frustrated because I tried three times.
"""

t2 = """Hi, I ordered a laptop last week but it still hasn’t been delivered.  
Customer support told me yesterday it would arrive today, but I just checked and the status still shows ‘processing’.  
This delay is really frustrating — I need the laptop urgently for my office work.
"""

t3 = """Hello, I received my new phone yesterday and it’s fantastic. 
The delivery was even faster than expected, and the packaging was perfect. 
I’m really satisfied with the whole experience.
"""

## Main App/Code

In [5]:
# Importing Libraries
import gradio as gr
import pandas as pd
import requests
import os
import json

# Initialize Groq 
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

headers = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}

def analyze_with_groq(transcript: str):
    url = "https://api.groq.com/openai/v1/chat/completions"

    prompt = f"""
    You are a call center assistant.
    Given this transcript:
    "{transcript}"

    Perform 2 tasks:
    1. Summarize in 1-2 sentences.
    2. Identify customer sentiment, choose only one(Positive, Neutral, Negative) 
    along with emotion(Angry, Frustrated, Disappointed, Happy, Grateful)
    and presnt as (Negative/Angry,Negative/Disappointed,Positive/Grateful etc)
    
    Return your answer strictly in this JSON format:
    {{
      "summary": "<summary here>",
      "sentiment": "<Positive/Neutral/Negative>"
    }}
    """

    data = {
        "model": "llama-3.1-8b-instant",  # fast Groq model
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code != 200:
        raise Exception(f"Groq API error: {response.status_code}, {response.text}")

    text = response.json()["choices"][0]["message"]["content"].strip()

    try:
        return json.loads(text)
    except:
        return {"summary": text, "sentiment": "Unknown"}


def analyze_and_save(transcript):
    if not transcript.strip():
        return "⚠️ Please enter a transcript.", "", ""

    result = analyze_with_groq(transcript)

    # Saving results into CSV
    df = pd.DataFrame(
        [[transcript, result["summary"], result["sentiment"]]],
        columns=["Transcript", "Summary", "Sentiment"]
    )
    CSV_FILE = "call_analysis.csv"
    if not os.path.exists(CSV_FILE):
        df.to_csv(CSV_FILE, index=False)
    else:
        df.to_csv(CSV_FILE, mode="a", header=False, index=False)

    return transcript, result["summary"], result["sentiment"]


#  Gradio UI 
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="green")) as demo:
    gr.Markdown(
        "<h1 style='text-align:center; color:#2563eb;'>📞 Call Transcript Analyzer</h1>"
        "<p style='text-align:center; color:#16a34a;'>Summarize conversations & detect sentiment instantly!</p>"
    )

    with gr.Row():
        transcript_in = gr.Textbox(
            label="Enter Call Transcript",
            placeholder="e.g. Hi, I tried to book a slot but payment failed...",
            lines=4
        )

    analyze_btn = gr.Button("🔍 Analyze Transcript", variant="primary")

    with gr.Row():
        transcript_out = gr.Textbox(label="Original Transcript", interactive=False)
        summary_out = gr.Textbox(label="Summary", interactive=False)
        sentiment_out = gr.Textbox(label="Sentiment", interactive=False)

    analyze_btn.click(
        analyze_and_save,
        inputs=[transcript_in],
        outputs=[transcript_out, summary_out, sentiment_out]
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7862
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://ae3ee6c3ffe8fb35f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Reading CSV File

In [6]:
import pandas as pd

csv = pd.read_csv('/kaggle/working/call_analysis.csv')
csv

,Transcript,Summary,Sentiment
0,"Hi, I ordered a laptop last week but it still ...",The customer is awaiting a laptop delivery tha...,Negative/Frustrated
1,"Agent: Hello, thank you for calling. How can I...",Customer was unable to complete a booking due ...,Negative/Frustrated
